COPY FROM:
https://cloud.tencent.com/developer/article/1651334

In [16]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, modify by wmathor
'''
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

dtype = torch.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3 words sentences (=sequence_length is 3)
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

# TextCNN Parameter
embedding_size = 50
sequence_length = len(sentences[0]) # every sentences contains sequence_length(=3) words
num_classes = 2  # 0 or 1
batch_size = 4

word_list = " ".join(sentences).split()
vocab = list(set(word_list))
word2idx = {w: i for i, w in enumerate(vocab)}
vocab_size = len(vocab)

def make_data(sentences, labels):
  inputs = []
  for sen in sentences:
      inputs.append([word2idx[n] for n in sen.split()])

  targets = []
  for out in labels:
      targets.append(out) # To using Torch Softmax Loss function
  return inputs, targets

input_batch, target_batch = make_data(sentences, labels)
input_batch, target_batch = torch.LongTensor(input_batch), torch.LongTensor(target_batch)

dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, batch_size, True)

class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.W = nn.Embedding(vocab_size, embedding_size)
        output_channel = 3
        self.conv = nn.Sequential(
            # conv : [input_channel(=1), output_channel, (filter_height, filter_width), stride=1]
            nn.Conv2d(1, output_channel, (2, embedding_size)),
            nn.ReLU(),
            # pool : ((filter_height, filter_width))
            nn.MaxPool2d((2, 1)),
        )
        # fc
        self.fc = nn.Linear(output_channel, num_classes)

    def forward(self, X):
      '''
      X: [batch_size, sequence_length]
      '''
      batch_size = X.shape[0]
      print(batch_size)
      embedding_X = self.W(X) # [batch_size, sequence_length, embedding_size]
      print(embedding_X.shape)
      embedding_X = embedding_X.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]
      print(embedding_X.shape)
      conved = self.conv(embedding_X) # [batch_size, output_channel, 1, 1]
      print(conved.shape)
      flatten = conved.view(batch_size, -1) # [batch_size, output_channel*1*1]
      print(flatten.shape)
      output = self.fc(flatten)
      return output

model = TextCNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training
for epoch in range(5):
  for batch_x, batch_y in loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    print(batch_x.shape)
    print(batch_x[0][0].dtype)
    print(batch_y.shape)
    print(batch_y[0].dtype)
    
    pred = model(batch_x)
    print(pred)
    loss = criterion(pred, batch_y)
    print(loss)
    break
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# Test
test_text = 'i hate me'
tests = [[word2idx[n] for n in test_text.split()]]
test_batch = torch.LongTensor(tests).to(device)
# Predict
model = model.eval()
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

torch.Size([4, 3])
torch.int64
torch.Size([4])
torch.int64
4
torch.Size([4, 3, 50])
torch.Size([4, 1, 3, 50])
torch.Size([4, 3, 1, 1])
torch.Size([4, 3])
tensor([[0.5547, 0.5734],
        [0.2099, 0.0853],
        [0.4629, 0.8694],
        [0.3475, 0.2044]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor(0.7503, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([4, 3])
torch.int64
torch.Size([4])
torch.int64
4
torch.Size([4, 3, 50])
torch.Size([4, 1, 3, 50])
torch.Size([4, 3, 1, 1])
torch.Size([4, 3])
tensor([[0.7559, 0.3582],
        [0.3475, 0.2044],
        [0.5547, 0.5734],
        [0.2099, 0.0853]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor(0.7489, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([4, 3])
torch.int64
torch.Size([4])
torch.int64
4
torch.Size([4, 3, 50])
torch.Size([4, 1, 3, 50])
torch.Size([4, 3, 1, 1])
torch.Size([4, 3])
tensor([[0.4629, 0.8694],
        [1.1867, 0.9314],
        [0.5547, 0.5734],
        [0.2099, 0.0853]], device='cuda:0', grad